In [5]:
# data wrangling imports
import numpy as np
import pandas as pd

In [7]:
# importing datasets
wolt = pd.read_csv("../geotracker/data/wolt_clean_data.csv").iloc[:, 1:]
liefe = pd.read_csv("../geotracker/data/lieferando_clean_data.csv")
maps = pd.read_csv("../geotracker/website/data/final.csv").drop(labels=3168)

# renaming columns
wolt.rename(columns={
    "type_of_cuisine_categorized": "type_of_cuisine",
    "avg_delivery_time_clean": "avg_delivery_time"
}, inplace=True)

liefe.rename(columns={"type_of_cuisine_categorized": "type_of_cuisine"}, inplace=True)

maps.rename(columns={
    'Coordinates': 'coordinates',
    'Name': 'restaurant_name',
    'Type': 'type',
    "Address": "address",
    "Opening Hours": "opening_hours",
    "Street": "street",
    "Bezirk": "city_name",
    "PLZ": "zip_code"
}, inplace=True)

# scaling lieferando
max_val = liefe[liefe.database == "lieferando"]["pricyness"].max()
liefe["pricyness"] = liefe["pricyness"].map(lambda x: 4 * (x - 0) / (max_val))

# converting zip_code to integer, in maps table
maps["zip_code"] = maps["zip_code"].astype('Int64')

# creating lat, lon columns for maps table
maps['latitude'] = maps.coordinates.apply(
    lambda x: x.replace("[", "").replace("]", "").split(", ")[0])

maps['longitude'] = maps.coordinates.apply(
    lambda x: x.replace("[", "").replace("]", "").split(", ")[1])

maps['coordinates'] = maps.coordinates.apply(
    lambda x: x.replace("[", "").replace("]", ""))

maps["database"] = "here_maps"

In [8]:
# concatenating datasets
all_deliveries_df = pd.concat([wolt,liefe])

# creating coordinates (lat,lon) column
all_deliveries_df["coordinates"] = all_deliveries_df.latitude.astype(
    str) + "," + all_deliveries_df.longitude.astype(str)

# creating address column 
all_deliveries_df[
    'address'] = all_deliveries_df.street + ", " + all_deliveries_df.zip_code.astype(
        str) + ", " + all_deliveries_df.city_name

# reorganizing tables
all_deliveries_df = all_deliveries_df[[
    'restaurant_name', 'avg_review_score', 'reviews', 'minimum_order_value',
    'delivery_fee', 'pricyness', 'avg_delivery_time', 'type_of_cuisine', 'address',
    'street', 'zip_code', 'city_name', 'coordinates', 'latitude', 'longitude',
    'database'
]]


In [9]:
# checking shapes
wolt.shape, liefe.shape, all_deliveries_df.shape, maps.shape

((1361, 13), (3330, 15), (4691, 16), (5461, 13))

In [10]:
# preprocessing type_of_cuisine
maps['Cuisine_1'] = maps.Cuisine_1.str.lower()

# dictionary containing all keywords and categorizing
unique_toc_dict = {
    'thai': 'asian',
    'fine-dining': np.nan,
    'cheese': np.nan,
    'fusion': np.nan,
    'butterchicken': 'indian',
    'chinese': 'asian',
    'traditional': np.nan,
    'café': 'cafes',
    'german': 'european',
    'cocktail': 'bars',
    'hamburger': 'american',
    'baklava': 'middle eastern',
    'pokebowl': 'poke',
    'bagel': 'breakfast/dessert',
    'waffles': 'cafes',
    'worklunch': 'fastfood',
    'tapas': 'mediterranean',
    'mediterranean': 'mediterranean',
    'pastries': 'breakfast/dessert',
    'sliders': 'fastfood',
    'turkish': 'middle eastern',
    'steak': 'steak',
    'snacks': 'snacks',
    'Sashimi': 'asian',
    'pancakes': 'breakfast/dessert',
    'Georgian': 'european',
    'risotto': 'italian',
    'glutenfree': 'healthy',
    'bento': 'asian',
    'bistro': 'european',
    'shawarma': 'middle eastern',
    'meatballs': 'european',
    'sushi': 'asian',
    'fish': 'seafood',
    'Don': np.nan,
    'bakery': 'breakfast/dessert',
    'delicious': np.nan,
    'mexican': 'mexican',
    'summerrolls': 'asian',
    'chickennuggets': 'fastfood',
    'korean': 'asian',
    'vegan': 'vegetarian or vegan',
    'chocolate': 'breakfast/dessert',
    'porridge': 'breakfast/dessert',
    'Schnitzel': 'european',
    'Arabic': 'middle eastern',
    'moussaka': 'greek',
    'Dessert': 'breakfast/dessert',
    'vegetarian': 'vegetarian or vegan',
    'donut': 'breakfast/dessert',
    'friedchicken': 'fastfood',
    'smoothie': 'breakfast/dessert',
    'beer': 'bars',
    'indian': 'indian',
    'rice': 'asian',
    'fruit': 'breakfast/dessert',
    'icecoffee': 'cafes',
    'Pastrami': np.nan,
    'dumplings': 'asian',
    'currywurst': 'fastfood',
    'asian': 'asian',
    'pizza': 'italian',
    'grill': 'steak',
    'seafood': 'seafood',
    'wine': 'bars',
    'schnitzel': 'european',
    'american': 'american',
    'healthy': 'healthy',
    'hummus': 'middle eastern',
    'russian': 'russian',
    'Donburi': "asian",
    'baguette': 'european',
    'salad': 'healthy',
    'fries': 'fastfood',
    'Austrian': 'european',
    'gyoza': 'asian',
    'potato': np.nan,
    'naan': 'indian',
    'icecream': 'breakfast/dessert',
    'pita': 'greek',
    'sausage': 'european',
    'neapolitanpizza': 'italian',
    'spaghetti': 'italian',
    'cake': 'breakfast/dessert',
    'ribs': 'american',
    'dessert': "breakfast/dessert",
    'Doughnut': 'breakfast/dessert',
    'matcha': 'cafes',
    'focaccia': 'italian',
    'homemade': np.nan,
    'milkshake': 'american',
    'taco': 'mexican',
    'curry': 'indian',
    'israeli': 'middle eastern',
    'bapburgersandpastrami': 'fastfood',
    'sandwich': 'fastfood',
    'bowl': 'poke',
    'maki': 'asian',
    'roll': 'asian',
    'brunch': 'breakfast/dessert',
    'pho': 'asian',
    'vietnamese': 'asian',
    'burgers': 'fastfood',
    'muchapizza': 'italian',
    'masala': 'indian',
    'contemporary': np.nan,
    'falafel': 'middle eastern',
    'gyros': 'greek',
    'friedrice': 'asian',
    'chicken': 'snacks',
    'italian': 'european',
    'spaetzle': 'european',
    'streetfood': 'fastfood',
    'Mozzarella': 'italian',
    'ramen': 'asian',
    'antipasti': 'italian',
    'noodles': 'italian',
    'hotdog': 'fastfood',
    'coffee': 'cafes',
    'oriental': 'middle eastern',
    'bubbletea': 'breakfast/dessert',
    'greek': 'greek',
    'middleeastern': 'middle eastern',
    'pasta': 'italian',
    'fresh': 'healthy',
    'tandoori': 'indian',
    'wrap': 'healthy',
    'european': 'european',
    'tea': 'breakfast/dessert',
    'bao': 'asian',
    'beyondmeat': 'vegetarian or vegan',
    'duck': 'asian',
    'galette': 'european',
    'panini': 'italian',
    'soup': 'healthy',
    'fastfood': 'fastfood',
    'LatinAmerican': 'south american',
    'juice': 'breakfast/dessert',
    'halal': 'middle eastern',
    'burger': 'american',
    'vegetable': 'vegetarian or vegan',
    'Hawaii': 'american',
    'breakfast': 'breakfast/dessert',
    'poke': 'poke',
    'drinks': 'bars',
    'burrito': 'mexican',
    'salmon': 'seafood',
    'homecooking': np.nan,
    'spanish': 'mediterranean',
    'meat': 'steak',
    'french': 'european',
    'veggieburger': 'vegetarian or vegan',
    'lunch': 'steak',
    'meze': 'middle eastern',
    'homemademeals': np.nan,
    'kebab': 'middle eastern',
    'beef': 'steak',
    'tex-mex': 'mexican',
    'japanese': 'asian',
    'arab': 'middle eastern',
    'italian style pizza': 'italian',
    'spanish/tapas': 'mediterranean',
    'drinks/snacks': 'bars',
    '100% halal': 'middle eastern',
    'bio': 'healthy',
    'ice cream': 'breakfast/dessert',
    'steaks': 'steak',
    'sandwiches': 'fastfood',
    'turkish pizza': 'middle eastern',
    'salads': 'healthy',
    'indonesian': 'asian',
    'german dishes': 'european',
    'döner': 'middle eastern',
    'argentinian': 'south american',
    'wraps': 'healthy',
    'desserts': 'breakfast/dessert',
    'spare ribs': 'american',
    'lebanese': 'middle eastern',
    'moroccan': 'middle eastern',
    'polish': 'european',
    'gluten-free': "healthy",
    'american style pizza': 'american',
    'austrian cuisine': 'european',
    'soups': 'healthy',
    'african': 'african',
    'other': np.nan,
    'balkans': 'european',
    'iranian': 'middle eastern',
    'baked goods': 'breakfast/dessert',
    '': np.nan,
    'balkan': 'european',
    'pakistani': np.nan,
    'middle eastern': 'middle eastern',
    'sicilian': 'italian',
    "international": "international",
    "brewpub": "bars",
    "chinese - cantonese": "asian",
    "austrian": "european",
    "hot dogs": "fastfood",
    "hawaiian/polynesian": "american",
    "dinner": np.nan,
    "canadian": "american",
    "chilean": 'south american',
    "japanese - sushi": "asian",
    "irish": "european",
    "argentinean": 'south american',
    "australian": "asian",
    "natural/healthy": "healthy",
    'caucasian': "european",
    'norwegian': "european",
    'hungarian': "european",
    'caribbean': 'south american',
    'american - creole': "american",
    'british': "european",
    'jewish/kosher': 'american',
    'crêperie': 'breakfast/dessert',
    'barbecue': 'steak',
    'fondue': "european",
    'american - cajun': "american",
    'bohemian': "european"
}

# in case we need to add more columns

# for x in liefe_df.type_of_cuisine.unique():
#     if x not in unique_toc_dict.keys():
#         unique_toc_dict[x] = np.nan

#to check if sth's missing
# for x in maps.Cuisine_1.unique():
#     if x not in unique_toc_dict:
#         print(x)

# updating type of cuisine columns
maps['type_of_cuisine'] = maps.Cuisine_1.map(unique_toc_dict)


# re-organizing columns
maps = maps[[
    'restaurant_name', 'type', 'type_of_cuisine', 'address',
    'street', 'zip_code', 'city_name', 'coordinates', 'latitude', 'longitude',
    'opening_hours', 'database']]


In [11]:
all_data_df = pd.concat([all_deliveries_df, maps])

In [12]:
all_data_df.drop(columns=[
    "avg_delivery_time", "delivery_fee", "minimum_order_value",
    "type"
],
              inplace=True)
all_data_df.drop_duplicates(inplace=True)

In [14]:
all_data_df

,restaurant_name,avg_review_score,reviews,pricyness,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,opening_hours
0,BunUp Mitte,7.8,NaN,1.0,vegetarian or vegan,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.39857,wolt,NaN
1,Machiavelli,8.4,NaN,2.0,european,"Albrechtstr.13 Berlin, 10117.0, Berlin",Albrechtstr.13 Berlin,10117.0,Berlin,"52.52130280012305,13.385180090564926",52.521303,13.38518,wolt,NaN
2,Pizza Peppino,NaN,NaN,1.0,european,"Tucholskystraße 13, 10117.0, Berlin",Tucholskystraße 13,10117.0,Berlin,"52.52339332586772,13.391764043640274",52.523393,13.391764,wolt,NaN
3,Fresh's,7.8,NaN,1.0,poke,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.39857,wolt,NaN
4,Bombay,8.4,NaN,2.0,middle eastern,"Friedrichstraße 106c, 10117.0, Berlin",Friedrichstraße 106c,10117.0,Berlin,"52.5232743,13.3884602",52.523274,13.38846,wolt,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5457,Cafe Seestern,NaN,NaN,NaN,european,Am Schlachtensee 70 Nikolassee 14129 Berlin,Am Schlachtensee 70,14129.0,Nikolassee,"52.44011, 13.21277",52.44011,13.21277,here_maps,NaN
5458,Gabana Restaurant,NaN,NaN,NaN,international,Breisgauer Straße 12 Zehlendorf 14129 Berlin,Breisgauer Straße 12,14129.0,Zehlendorf,"52.43783, 13.2154",52.43783,13.2154,here_maps,NaN
5459,Osteria Carlo,NaN,NaN,NaN,european,Breisgauer Straße 14 Zehlendorf 14129 Berlin,Breisgauer Straße 14,14129.0,Zehlendorf,"52.43762, 13.2155",52.43762,13.2155,here_maps,NaN
5460,Pizza Piazza,NaN,NaN,NaN,NaN,Bülowstraße 1 Zehlendorf 14163 Berlin,Bülowstraße 1,14163.0,Zehlendorf,"52.43707, 13.23353",52.43707,13.23353,here_maps,NaN


In [15]:
# saving de into csv
all_deliveries_df.to_csv("../geotracker/data/deliveries_data.csv")
maps.to_csv("../geotracker/data/maps_clean_data.csv")
all_data_df.to_csv("../geotracker/data/all_data.csv")